In [18]:
import nltk
from nltk.tokenize import word_tokenize

from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import LSTM
from keras.layers.embeddings import Embedding

In [20]:
# Artificial Dataset
corpus = [
    # Positive Reviews
    'This is an excellent movie',
    'The move was fantastic I like it',
    'You should watch it is brilliant',
    'Exceptionally good',
    'Wonderfully directed and executed I like it',
    'Its a fantastic series',
    'Never watched such a brillent movie',
    'It is a Wonderful movie',

    # Negtive Reviews
    "horrible acting",
    'waste of money',
    'pathetic picture',
    'It was very boring',
    'I did not like the movie',
    'The movie was horrible',
    'I will not recommend',
    'The acting is pathetic'
]
sentiments = array([1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0])

In [ ]:
def load_file(file_name):
    with open(file_name, 'r', encoding='utf-8') as f:
        conversations = f
    return conversations
    
corpus = load_file("data/train/formatted_single_dialogues_train.txt")
sentiments = load_file("data/train/formatted_single_dialogues_emotion_train.txt")

In [22]:
nltk.download('punkt')
vocab_length = 50

all_words = []
for sent in corpus:
    tokenize_word = word_tokenize(sent)
    for word in tokenize_word:
        all_words.append(word)

unique_words = set(all_words)

embedded_sentences = [one_hot(sent, vocab_length) for sent in corpus]

word_count = lambda sentence: len(word_tokenize(sentence))
longest_sentence = max(corpus, key=word_count)
length_long_sentence = len(word_tokenize(longest_sentence))

padded_sentences = pad_sequences(embedded_sentences, length_long_sentence, padding='post')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [42]:
model = Sequential()
model.add(Embedding(vocab_length, 20, input_length=length_long_sentence))
#model.add(LSTM(64))

#model.add(Conv1D(128, 5, activation='relu'))
#model.add(GlobalMaxPooling1D())

model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
print(model.summary())

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 7, 20)             1000      
_________________________________________________________________
flatten (Flatten)            (None, 140)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 141       
Total params: 1,141
Trainable params: 1,141
Non-trainable params: 0
_________________________________________________________________
None


In [43]:
model.fit(padded_sentences, sentiments, epochs=50, verbose=1)

Epoch 1/50
1/1 [==============================] - 1s 523ms/step - loss: 0.6931 - acc: 0.5000
Epoch 2/50
1/1 [==============================] - 0s 8ms/step - loss: 0.6898 - acc: 0.5625
Epoch 3/50
1/1 [==============================] - 0s 6ms/step - loss: 0.6865 - acc: 0.6875
Epoch 4/50
1/1 [==============================] - 0s 5ms/step - loss: 0.6833 - acc: 0.6875
Epoch 5/50
1/1 [==============================] - 0s 11ms/step - loss: 0.6800 - acc: 0.6875
Epoch 6/50
1/1 [==============================] - 0s 8ms/step - loss: 0.6767 - acc: 0.8750
Epoch 7/50
1/1 [==============================] - 0s 5ms/step - loss: 0.6735 - acc: 0.9375
Epoch 8/50
1/1 [==============================] - 0s 5ms/step - loss: 0.6702 - acc: 1.0000
Epoch 9/50
1/1 [==============================] - 0s 7ms/step - loss: 0.6670 - acc: 1.0000
Epoch 10/50
1/1 [==============================] - 0s 16ms/step - loss: 0.6637 - acc: 1.0000
Epoch 11/50
1/1 [==============================] - 0s 5ms/step - loss: 0.6605 - acc: 

In [44]:
loss, accuracy = model.evaluate(padded_sentences, sentiments, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 93.750000
